<a href="https://colab.research.google.com/github/emerenan/analysing_police_activity/blob/master/police_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd

# Método para leitura dos dados
def readfile(path):
    """ Método para ler os dados do arquivo e retornar um dataframe.
    Args: \n
        path: localização do arquivo que será lido. \n

    Returns: \n
        Dataframe contendo os dados para analise.

    Raises: \n
        ParserError: retorna as linhas que não puderam ser lidas.
    """

    try:
        df = pd.read_csv(path, error_bad_lines=False, warn_bad_lines=True, parse_dates=True, infer_datetime_format=True)
        """
        print(df.head())
        print('-----------------------------------------------------------------------')
        print(df.isnull().sum())
        print('-----------------------------------------------------------------------')
        print(df.info())
        print('-----------------------------------------------------------------------')
        """
        return df
    except pd.io.common.ParserError:
        print("Dados não puderam ser lidos")

# lendo o arquivo e salvando na variavel data
data = readfile('/content/police.csv')

dict = {}
for label, entry in data.isnull().sum().items():
    # Salvando a forma do dataframe na variavel t
    t = data.shape
    # Salvando a qtd de linhas na variavel rows
    rows = t[0]

    if entry == rows or entry >= rows/2:
        data.drop(label, axis='columns', inplace=True)
        if label in dict.keys():
            dict[label] += "Deletada"
        else:
            dict[label] = "Deletada"
"""
for key, value in dict.items():
    if value == 0:
        print("Não houve colunas deletadas!")
    else:
        print(key, ": ", value)
"""
print('Apagando colunas null!')
print('-----------------------------------------------------------------------')
# Apagando todas as linhas do dataframe, com base nas linhas NUll da coluna driver_gender
print(data.shape) #antes de apagar
data.dropna(subset=['driver_gender'], inplace=True)
print(data.shape) #depois de apagado
print('Apagando linhas Null com base na coluna driver_gender!')
print('-----------------------------------------------------------------------')

# Analisando os tipos de dados do Dataframe para alterar o dtype
data.is_arrested = data.is_arrested.astype('bool')

# combinando stop_date com stop_time e convertendo para string
dates = data.stop_date.str.cat(data.stop_time, sep=' ')

# Criando a coluna stop_datetime e convertendo os valores para datetime
data['stop_datetime'] = pd.to_datetime(dates)

# Trocando a coluna stop_datetime como index do dataframe
data.set_index('stop_datetime', inplace=True)

print('Tratamento de tipos de dados do Dataframe.')
print('-----------------------------------------------------------------------')

# Contabilizando o resultado das abordagens policiais em porcentagem
data.stop_outcome.value_counts(normalize=True)

# Contabilizando os tipos de motoristas
data.driver_race.value_counts()

# Criação de variaveis para cada tipo de motorista
white = data[data.driver_race == "White"]
black = data[data.driver_race == "Black"]
hispanic = data[data.driver_race == "Hispanic"]
asian = data[data.driver_race == "Asian"]
others = data[data.driver_race == "Other"]

# Comparando os resultados das abordagens por grupo
print("Abordagens em Brancos")
#variavel.['coluna_do_df'].values_count(normalize=True)
print(white.stop_outcome.value_counts(normalize=True))
print("\n")
print("Abordagens em Negros")
print(black.stop_outcome.value_counts(normalize=True))
print("\n")

# criação de variaveis para cada gênero de motorista
male = data[data.driver_gender == 'M']
female = data[data.driver_gender == 'F']

# Comparando os resultados das voiolações por genero
print("Violações do gênero Masculino")
print(male.violation.value_counts(normalize=True))
print('\n')
print("Violações do gênero Feminino")
print(female.violation.value_counts(normalize=True))
print('\n')

# Criação de variavel para filtro por genero e violação de acesso de velocidade
male_arrested = data[(data.driver_gender == 'M') & (data.violation == 'Speeding')]
female_arrested = data[(data.driver_gender == 'F') & (data.violation == 'Speeding')]

# Comparando os resultados das voiolações
print("Excesso velocidade por homens presos")
print(male_arrested.stop_outcome.value_counts(normalize=True))
print('\n')
print("Excesso velocidade por mulheres presa")
print(female_arrested.stop_outcome.value_counts(normalize=True))
print('\n')

# comparação booleana das prisões (is_arrested) por genero e distrito
data[data.district == 'Zone K1'].is_arrested.mean() # taxa na zona K1

data.groupby(by=['district', 'driver_gender']).is_arrested.mean()

# Comparação da pesquisa de conduta por violação e genero do motorista
print(data.groupby(by=['violation', 'driver_gender']).search_conducted.mean())

Apagando colunas null!
-----------------------------------------------------------------------
(91741, 13)
(86536, 13)
Apagando linhas Null com base na coluna driver_gender!
-----------------------------------------------------------------------
Tratamento de tipos de dados do Dataframe.
-----------------------------------------------------------------------
Abordagens em Brancos
Citation            0.902263
Warning             0.057508
Arrest Driver       0.024018
No Action           0.007031
N/D                 0.006433
Arrest Passenger    0.002748
Name: stop_outcome, dtype: float64


Abordagens em Negros
Citation            0.857224
Warning             0.065120
Arrest Driver       0.054294
N/D                 0.008547
Arrest Passenger    0.008303
No Action           0.006512
Name: stop_outcome, dtype: float64


Violações do gênero Masculino
Speeding               0.522243
Moving violation       0.206144
Equipment              0.134158
Other                  0.058985
Registration/pla